<a href="https://colab.research.google.com/github/moonsunglab/test/blob/main/1/moonsung.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import pandas as pd
import numpy as np
import folium
from folium.plugins import HeatMap
from shapely.geometry import box
import geopandas as gpd
from collections import defaultdict

# 로드킬

## 로드킬 데이터 생성

In [ ]:
file_path = 'https://github.com/LeeMinJun0102/test/raw/refs/heads/main/README.md'

In [ ]:
rk_df = pd.read_csv(file_path, sep='\t', names=['number', 'road_kill', 'address', '날짜', '시간', '지역', '경도', '위도'], skiprows=1)
# 마지막 행 제거
rk_df = rk_df[:-1]
# 연도만 담고 있는 파생변수 생성
rk_df['연도'] = rk_df['날짜'].str.split('-').str[0]

In [ ]:
rk_df

,number,road_kill,address,날짜,시간,지역,경도,위도,연도
0,3974,로드킬,경기 남양주시 와부읍 덕소리 392-2 번지에서 로드킬이 발생하였습니다.,2020-12-31,18:41,경기 남양주시,127.214314,37.593123,2020
1,3973,로드킬,세종특별자치시 연동면 합강리 526-11 번지에서 로드킬이 발생하였습니다.,2020-12-31,18:17,세종특별자치시 세종시,127.324362,36.528559,2020
2,3972,로드킬,세종특별자치시 연동면 합강리 96-13 번지에서 로드킬이 발생하였습니다.,2020-12-31,18:16,세종특별자치시 세종시,127.328889,36.524532,2020
3,3971,로드킬,대전 대덕구 미호동 1-10 번지에서 로드킬이 발생하였습니다.,2020-12-31,18:10,대전 대덕구,127.479598,36.475791,2020
4,3970,로드킬,세종특별자치시 가람동 245-5 번지에서 로드킬이 발생하였습니다.,2020-12-31,18:00,세종특별자치시 세종시,127.238288,36.470593,2020
...,...,...,...,...,...,...,...,...,...
9476,5,로드킬,충남 청양군 청양읍 읍내리 183-4 번지에서 로드킬이 발생하였습니다.,2022-01-01,10:15,충남 청양군,126.800164,36.449393,2022
9477,4,로드킬,강원 강릉시 포남동 1096-8 번지에서 로드킬이 발생하였습니다.,2022-01-01,10:06,강원 강릉시,128.904923,37.770272,2022
9478,3,로드킬,경기 용인시 기흥구 보정동 1019-1003 번지에서 로드킬이 발생하였습니다.,2022-01-01,08:09,경기 용인시 기흥구,127.098973,37.308128,2022
9479,2,로드킬,경기 용인시 기흥구 서천동 733 번지에서 로드킬이 발생하였습니다.,2022-01-01,06:24,경기 용인시 기흥구,127.071588,37.240608,2022


## 로드킬 그리드 생성

In [ ]:
# 1. 위도/경도 범위
min_lat, max_lat = 33.00, 38.50
min_lon, max_lon = 126.00, 130.00

# 2. 격자 크기 설정 (0.1도 ≈ 10~11km)
grid_size = 0.3

# 3. 격자 생성
grid_cells = []
grid_ids = []
grid_id = 1

lon = min_lon
while lon < max_lon:
    lat = min_lat
    while lat < max_lat:
        cell = box(lon, lat, lon + grid_size, lat + grid_size)
        grid_cells.append(cell)
        grid_ids.append(f'grid_{grid_id}')
        grid_id += 1
        lat += grid_size
    lon += grid_size

# 4. GeoDataFrame으로 변환
grid = gpd.GeoDataFrame({'grid_id': grid_ids, 'geometry': grid_cells}, crs='EPSG:4326')

## 그리드에 로드킬 빈도수 넣기(dict)

In [ ]:
# 결과 저장용 딕셔너리: grid_id → 포함된 좌표 개수
grid_counts = defaultdict(int)
grid_list = []

# df에 '위도', '경도' 열이 있다고 가정
for i, row in rk_df.iterrows():  # i는 인덱스, row는 한 줄
    lat = row['위도']
    lon = row['경도']

    # 각 격자(box) 범위 안에 포함되는지 확인
    for grid_id, cell in zip(grid_ids, grid_cells):
        minx, miny, maxx, maxy = cell.bounds
        if miny <= lat < maxy and minx <= lon < maxx:
            grid_counts[grid_id] += 1
            grid_list.append(grid_id)
            break  # 이미 해당하는 격자 찾았으면 멈춰

In [ ]:
# 확인용
# grid_counts

In [ ]:
# 원하는 그리드 넘버
list_key = []   #> 그리드 넘버
list_val = []   #> 그리드 넘버에 해당하는 로드킬 수
for key, val in grid_counts.items():
    if val <= 10: #> 로드킬 수 n회 이하
        list_key.append(key)
        list_val.append(val)

In [ ]:
# grid 파생 컬럼 추가
rk_df['grid'] = grid_list

In [ ]:
rk_df = rk_df[['grid', '경도', '위도', '연도']]

In [ ]:
rk_df['grid'].value_counts()

,count
grid,
grid_73,834
grid_72,726
grid_54,587
grid_71,528
grid_53,513
...,...
grid_203,2
grid_62,1
grid_100,1


## 로드킬 map 그리기(mp)

In [ ]:
map_center = [37.5665, 126.9780]
mp = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

In [ ]:
for grid_num in list_key[:5]:
    for idx, row in rk_df[rk_df['grid'] == grid_num].iterrows():
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=row.get('name', f"위도: {row['위도']}, 경도: {row['경도']}"),
            tooltip=row.get('name', '포인트'),
        icon=folium.Icon(color='black', icon='info-sign')).add_to(mp)

In [ ]:
mp

# 생태 통로

## 생태 통로 데이터 생성

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
file_path = '/content/drive/MyDrive/중간 프로젝트/장소(생태통로) 예측 자료/생태통로 시설정보 목록.xlsx'

In [ ]:
# eco road?
er_df = pd.read_excel(file_path)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/중간 프로젝트/장소(생태통로) 예측 자료/생태통로 시설정보 목록.xlsx'

In [ ]:
er_df = er_df[['생태통로 유형', '관리기관', '위도', '경도']]

In [ ]:
er_df

## 생태 통로 그리드 생성

In [ ]:
# 1. 위도/경도 범위
min_lat2, max_lat2 = 33.00, 38.50
min_lon2, max_lon2 = 126.00, 130.00

# 2. 격자 크기 설정 (0.1도 ≈ 10~11km)
grid_size2 = 0.3

# 3. 격자 생성
grid_cells2 = []
grid_ids2 = []
grid_id2 = 1

lon2 = min_lon2
while lon2 < max_lon2:
    lat2 = min_lat2
    while lat2 < max_lat2:
        cell2 = box(lon2, lat2, lon2 + grid_size2, lat2 + grid_size2)
        grid_cells2.append(cell2)
        grid_ids2.append(f'grid_{grid_id2}')
        grid_id2 += 1
        lat2 += grid_size2
    lon2 += grid_size2

# 4. GeoDataFrame으로 변환
grid2 = gpd.GeoDataFrame({'grid_id': grid_ids2, 'geometry': grid_cells2}, crs='EPSG:4326')

## 그리드에 생태 통로 빈도수 넣기(dict)

In [ ]:
# 결과 저장용 딕셔너리: grid_id → 포함된 좌표 개수
grid_counts2 = defaultdict(int)
grid_list2 = []

# df에 '위도', '경도' 열이 있다고 가정
for i, row in er_df.iterrows():  # i는 인덱스, row는 한 줄
    lat2 = row['위도']
    lon2 = row['경도']

    # 각 격자(box) 범위 안에 포함되는지 확인
    for grid_id2, cell2 in zip(grid_ids2, grid_cells2):
        minx, miny, maxx, maxy = cell2.bounds
        if miny <= lat2 < maxy and minx <= lon2 < maxx:
            grid_counts2[grid_id2] += 1
            grid_list2.append(grid_id2)
            break  # 이미 해당하는 격자 찾았으면 멈춰

In [ ]:
# 원하는 그리드 넘버
list_key2 = []   #> 그리드 넘버
list_val2 = []   #> 그리드 넘버에 해당하는 로드킬 수
for key, val in grid_counts2.items():
    if val <= 10: #> 로드킬 수 n회 이하
        list_key.append(key)
        list_val.append(val)

In [ ]:
# grid 파생 컬럼 추가
er_df['grid'] = grid_list2

In [ ]:
er_df

## 생태 통로 map 그리기(mp2) -> 로드킬이랑 같이 볼거면 mp로 바꾸기

In [ ]:
map_center2 = [37.5665, 126.9780]
mp2 = folium.Map(location=map_center, zoom_start=12, tiles='OpenStreetMap')

In [ ]:
for grid_num in list_key[:5]:
    for idx, row in er_df[er_df['grid'] == grid_num].iterrows():
        folium.Marker(
            location=[row['위도'], row['경도']],
            popup=row.get('name', f"위도: {row['위도']}, 경도: {row['경도']}"),
            tooltip=row.get('name', '포인트'),
        icon=folium.Icon(color='black', icon='info-sign')).add_to(mp2)

In [ ]:
mp2

# 그리드 별 생태 통로 수, 로드킬 수 DataFrame

In [ ]:
df = pd.DataFrame()

In [ ]:
grid_list = []
for i in range(251):
    name = f'grid_{i}'
    grid_list.append(name)

In [ ]:
df['그리드'] = grid_list

In [ ]:
df

## 로드킬 grid_counts dict -> DataFrame

In [ ]:
rk_dict_df = pd.DataFrame()

In [ ]:
rk_dict_df['그리드'] = grid_counts.keys()
rk_dict_df['로드킬 수'] = grid_counts.values()

In [ ]:
rk_dict_df

## 생태통로 grid_counts2 dict -> DataFrame

In [ ]:
er_dict_df = pd.DataFrame()

In [ ]:
er_dict_df['그리드'] = grid_counts2.keys()
er_dict_df['생태통로 수'] = grid_counts2.values()

In [ ]:
er_dict_df

## tot_df --> df, rk_dict_df, er_dict_df 합치기

In [ ]:
tot_df = df.merge(rk_dict_df, how='left').merge(er_dict_df, how='left')

In [ ]:
tot_df.sort_values(by='로드킬 수', ascending=False)

In [ ]:
tot_df.info()

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box

# 1. 남한 위도/경도 범위
min_lat, max_lat = 33.0, 38.5
min_lon, max_lon = 126.0, 130.0

# 2. 6x20 그리드
rows, cols = 6, 20
lat_step = (max_lat - min_lat) / rows
lon_step = (max_lon - min_lon) / cols

# 3. 사각형 폴리곤 생성
grid_cells = []
for i in range(rows):
    for j in range(cols):
        lat1 = min_lat + i * lat_step
        lat2 = lat1 + lat_step
        lon1 = min_lon + j * lon_step
        lon2 = lon1 + lon_step
        grid_cells.append(box(lon1, lat1, lon2, lat2))

grid = gpd.GeoDataFrame({'geometry': grid_cells})

# 4. 지도(배경)와 함께 시각화
fig, ax = plt.subplots(figsize=(6, 8))

# 남한 shapefile이 있으면 추가 가능 (없으면 grid만 그림)
# korea = gpd.read_file("KOR_adm0.shp")
# korea.plot(ax=ax, color='white', edgecolor='black')

grid.boundary.plot(ax=ax, color="red", linewidth=0.5)
plt.title("남한 위 6x20 그리드")
plt.show()
